In [8]:
import os
import time
import pandas as pd
from bs4 import BeautifulSoup, Tag
from datetime import datetime
from pytz import timezone
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from typing import Dict, Union
from urllib import request
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException

In [9]:
def launch_driver(driver_path: str):
    options = webdriver.ChromeOptions()
    options.add_argument('no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    # Set screen size to 1080p
    options.add_argument('--window-size=1920,1080')
    options.add_experimental_option(
        'excludeSwitches', ['enable-logging'])
    options.headless = False
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [10]:
path = ChromeDriverManager(path=os.getcwd()).install()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\HP\OneDrive\Desktop\Bihar_web_Scrapper\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


In [11]:
url = "https://www.eproc.bihar.gov.in"

In [12]:
driver = launch_driver(path)

In [13]:
driver.get(url)

In [15]:
# Close the information banner
close_btn_class_name = 'sha-pg010-close'
WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CLASS_NAME, close_btn_class_name)))
time.sleep(2)
driver.find_element(By.CLASS_NAME, close_btn_class_name).click()

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x0039B8F3+2406643]
	Ordinal0 [0x0032AF31+1945393]
	Ordinal0 [0x0021C748+837448]
	Ordinal0 [0x002492E0+1020640]
	Ordinal0 [0x0024957B+1021307]
	Ordinal0 [0x00276372+1205106]
	Ordinal0 [0x002642C4+1131204]
	Ordinal0 [0x00274682+1197698]
	Ordinal0 [0x00264096+1130646]
	Ordinal0 [0x0023E636+976438]
	Ordinal0 [0x0023F546+980294]
	GetHandleVerifier [0x00609612+2498066]
	GetHandleVerifier [0x005FC920+2445600]
	GetHandleVerifier [0x00434F2A+579370]
	GetHandleVerifier [0x00433D36+574774]
	Ordinal0 [0x00331C0B+1973259]
	Ordinal0 [0x00336688+1992328]
	Ordinal0 [0x00336775+1992565]
	Ordinal0 [0x0033F8D1+2029777]
	BaseThreadInitThunk [0x76816739+25]
	RtlGetFullPathName_UEx [0x77E98AFF+1215]
	RtlGetFullPathName_UEx [0x77E98ACD+1165]


In [ ]:
# Click on the Tenders link and then switch to the tab
# Close the previous tab as we don't need that
driver.find_element(By.CLASS_NAME, 'sha-pg001-02-menu-item-title-icon').click()
driver.close()
driver.switch_to.window(driver.window_handles[-1])

In [ ]:
def clean_text(text: str):
    text = ' '.join(filter(lambda x: x, text.replace(
        '\n', ' ').replace('\t', ' ').replace(':', '').split()))
    if text == '':
        return None
    return text.strip()

def format_key(text: str):
    return clean_text(text).replace('.', '').replace(' ', '_').replace('/', '').replace('₹', 'rs').replace('(', '_').replace(')', '').replace('__', '_').lower().strip()

In [ ]:
# Loop through the pages
def next_page():
    paginate = driver.find_element(By.CLASS_NAME, 'paginationLinks')
    page_links = list(filter(lambda x: 'page' in x.get_attribute('class'), paginate.find_elements(By.TAG_NAME, 'a')))
    for page_link in page_links:
        if page_link.get_attribute('class') == 'page current':
            try:
                nxt_pg = page_links[page_links.index(page_link) + 1]
                driver.execute_script('arguments[0].scrollIntoView();', nxt_pg)
                return nxt_pg
            except IndexError:
                pass

In [ ]:
# Loop through the tenders in a page
def tender_link():
    t_links = driver.find_element(By.ID, 'tblSummary').find_elements(By.CLASS_NAME, 'alink')
    try:
        link = t_links[link_index]
        driver.execute_script('arguments[0].scrollIntoView();', link)
        return link
    except IndexError:
        pass

In [ ]:
df = pd.DataFrame()

scraped_data = {}
while True:
    link_index = 0
    while tender_link():
        main_window = driver.current_window_handle
        tender_link().click()
        link_index += 1
        driver.switch_to.window(driver.window_handles[-1])

        # Wait till the tender detail loads up
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, 'tenderDetail')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tender_detail = soup.find('table', {'id': 'tenderDetail'})
        dept_name = soup.find('div', {'class': 'info'}).find('td').text
        data = {'department': dept_name}
        is_test_tender = False

        # This part scrapes all the tender data as key:value pair
        for i in tender_detail.find_all('tr', {'class': 'odd'}):
            for j in i.find_all('td', {'class': 'right b'}):
                key = format_key(list(filter(lambda x: x is not None and 'class' not in x.attrs, j.find_all('span')))[0].text)
                val = clean_text(j.find_next('td').text)
                if key == 'cot' and val.lower() == 'test':
                    # Add a flag for skipping test tenders
                    is_test_tender = True
                data[key] = val
        
        if not is_test_tender:
            df = df.append(data, ignore_index=True)

        driver.close()
        driver.switch_to.window(main_window)

    if next_page():
        next_page().click()
    else:
        break

In [ ]:
print(df.columns)
df

In [ ]:
# df.to_excel('bihar_tenders.xlsx')